# Extracting data from SQL Server

In [1]:
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
from time import time

In [2]:
from sqlalchemy import create_engine

user = 'basit'
password = 'test123'
server = 'localhost'
database = 'ETLProjectDB'

# Replace 'ODBC Driver 17 for SQL Server' with the appropriate ODBC driver name if needed
engine = create_engine(f'mssql+pyodbc://{user}:{password}@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')

try:
    with engine.connect() as conn:
        print(f"Successfully connected to the database: {database}")
except Exception as ex:
    print(f"Sorry, could not connect to the database: {ex}")

Successfully connected to the database: ETLProjectDB


In [3]:
# Extracting table names present in the database
table_names = engine.table_names()
print(table_names)

['greentrip_table', 'zone_table']


C:\Users\92322\AppData\Local\Temp\ipykernel_11108\571788782.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [4]:
dataframes = {}

for table_name in table_names:
    sql_query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(sql_query, engine)
    # Store the DataFrame in the dictionary using the table name as the key
    dataframes[table_name + '_df'] = df

In [5]:
dataframes.keys()

dict_keys(['greentrip_table_df', 'zone_table_df'])

# Transforming Data

In [6]:
greentrip_df = dataframes['greentrip_table_df']
zone_df      = dataframes['zone_table_df']

In [7]:
greentrip_df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,None,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,None,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,None,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,None,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,None,0.3,19.30,2,1,NaN


In [8]:
greentrip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 630918 entries, 0 to 630917
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               630918 non-null  int64  
 1   lpep_pickup_datetime   630918 non-null  object 
 2   lpep_dropoff_datetime  630918 non-null  object 
 3   store_and_fwd_flag     630918 non-null  object 
 4   RatecodeID             630918 non-null  int64  
 5   PULocationID           630918 non-null  int64  
 6   DOLocationID           630918 non-null  int64  
 7   passenger_count        630918 non-null  int64  
 8   trip_distance          630918 non-null  float64
 9   fare_amount            630918 non-null  float64
 10  extra                  630918 non-null  float64
 11  mta_tax                630918 non-null  float64
 12  tip_amount             630918 non-null  float64
 13  tolls_amount           630918 non-null  float64
 14  ehail_fee              0 non-null   

In [9]:
# converting lpep_pickup_datetime, lpep_dropoff_datetime in datetime datatype
greentrip_df.lpep_pickup_datetime = pd.to_datetime(greentrip_df['lpep_pickup_datetime'])
greentrip_df.lpep_dropoff_datetime = pd.to_datetime(greentrip_df['lpep_dropoff_datetime'])
print(greentrip_df['lpep_pickup_datetime'].dtype)
print(greentrip_df['lpep_dropoff_datetime'].dtype)

datetime64[ns]
datetime64[ns]


In [10]:
greentrip_df['ehail_fee'].isnull().count()

630918

In [11]:
greentrip_df.drop(columns=['ehail_fee'], inplace=True)

In [12]:
greentrip_df[greentrip_df['tolls_amount'] >= 1].count()

VendorID                 26034
lpep_pickup_datetime     26034
lpep_dropoff_datetime    26034
store_and_fwd_flag       26034
RatecodeID               26034
PULocationID             26034
DOLocationID             26034
passenger_count          26034
trip_distance            26034
fare_amount              26034
extra                    26034
mta_tax                  26034
tip_amount               26034
tolls_amount             26034
improvement_surcharge    26034
total_amount             26034
payment_type             26034
trip_type                26034
congestion_surcharge      3652
dtype: int64

# Loading data into PostgreSQL Database

In [13]:
pg_user = 'postgres'
pg_pass = 'mysecretpassword'
pg_host = 'localhost'
pg_port = 5432
pg_db   = 'ETLProjectDB'
engine = create_engine(f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}')
con = engine.connect()

In [14]:
start_time = time()
greentrip_df.to_sql(name='greentrip', con=con, index=False, if_exists='replace')
end_time = time()
diff = format(end_time - start_time)
print("Time : ",round(float(diff),2), " seconds") 

Time :  155.24  seconds


In [15]:
# Close the SQLAlchemy connection
con.close()